In [1]:
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display

In [2]:
display(pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', nrows=2).head())

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,31845504,10/28/2015 03:24:21 AM,NaN,NYPD,NYPD,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11219,1101 OVINGTON AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.626612,-74.007500,"(40.62661172175207, -74.00750037666796)"
1,31839078,10/28/2015 03:23:44 AM,10/28/2015 03:23:55 AM,DHS,Operations Unit - Department of Homeless Services,Homeless Person Assistance,No Status Call,NaN,10019,712 5 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.762495,-73.975193,"(40.76249450483886, -73.9751932073919)"


In [3]:
disk_engine = create_engine('sqlite:///311_8M.db')

In [ ]:
start = dt.datetime.now()
chunksize = 20000
j = 0
index_start = 1

for df in pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) # Remove spaces from columns

    df['CreatedDate'] = pd.to_datetime(df['CreatedDate']) # Convert to datetimes
    df['ClosedDate'] = pd.to_datetime(df['ClosedDate'])

    df.index += index_start

    # Remove the un-interesting columns
    columns = ['Agency', 'CreatedDate', 'ClosedDate', 'ComplaintType', 'Descriptor',
               'CreatedDate', 'ClosedDate', 'TimeToCompletion',
               'City']

    for c in df.columns:
        if c not in columns:
            df = df.drop(c, axis=1)    

    
    j+=1
    print '{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize)

    df.to_sql('data', disk_engine, if_exists='append')
    index_start = df.index[-1] + 1

In [ ]:
df.head()